# Strands Agents with Bedrock AgentCore Code Interpreter

本研讨会演示如何将 Strands Agents 与 Amazon Bedrock AgentCore Code Interpreter 集成，以创建能够动态执行代码的 AI 代理。

## 概述

在本实验中，您将：
- 了解 Bedrock AgentCore Code Interpreter 功能
- 使用 Strands Agents 测试默认 Code Interpreter
- 创建具有公共网络访问权限的自定义 Code Interpreter

## 先决条件

开始本实验之前，请确保您具备：
- 已配置 AWS 凭证（IAM 角色或环境变量）
- 已安装所需的 Python 包
- 对 Strands Agents 和 Bedrock AgentCore 概念有基本了解

如果您没有在假设 IAM 角色的环境中运行，请将您的 AWS 凭证设置为环境变量：

In [ ]:
import os

#os.environ["AWS_ACCESS_KEY_ID"]=<YOUR ACCESS KEY>
#os.environ["AWS_SECRET_ACCESS_KEY"]=<YOUR SECRET KEY>
#os.environ["AWS_SESSION_TOKEN"]=<OPTIONAL - YOUR SESSION TOKEN IF TEMP CREDENTIAL>
#os.environ["AWS_REGION"]=<AWS REGION WITH BEDROCK AGENTCORE AVAILABLE>

为 Strands Agents 和 Bedrock AgentCore Python SDK 安装所需的包：

In [ ]:
#%pip install -q strands-agents strands-agents-tools bedrock-agentcore

## 什么是 Bedrock AgentCore Code Interpreter？

Amazon Bedrock AgentCore Code Interpreter 是一个强大的工具，允许 AI 代理在安全的沙盒环境中动态执行代码。主要优势包括：

- **安全执行**：在隔离的沙盒环境中运行 Python 代码
- **动态问题解决**：使代理能够执行计算、分析数据和生成可视化
- **灵活配置**：支持默认沙盒和自定义网络启用环境
- **集成就绪**：与 Strands Agents 和其他 AI 框架无缝集成

Code Interpreter 为代理提供了解决需要计算分析、数据处理或数学计算的复杂问题的能力。

### 使用默认 Code Interpreter 测试 Strands Agent

让我们首先使用默认的 AgentCore Code Interpreter 测试 Strands Agent。我们将演示它如何在沙盒环境中生成和执行代码来解决数学问题。

In [ ]:
from strands import Agent
from strands_tools.code_interpreter import AgentCoreCodeInterpreter

agentcore_code_interpreter = AgentCoreCodeInterpreter()

# Create a code gen assistant agent
agent = Agent(
    model="us.amazon.nova-pro-v1:0",  # Optional: Specify the model ID
    system_prompt="""You are a helpful AI assistant that validates all answers through code execution.""",
    #system_prompt="""您是一位乐于助人的人工智能助手，可以通过代码执行来验证所有答案。""",
    tools=[agentcore_code_interpreter.code_interpreter],
)

#response = agent("Tell me the largest random prime number between 1 and 100, which is less than 84 and more that 9")
response = agent("告诉我 1 到 100 之间最大的随机素数，小于 84 且大于 9")
print(response)

让我们检查代理循环的详细执行流程，以了解代理如何处理请求和生成响应：

In [ ]:
print("Agent Loop Detail")
print("-----------------")

print(f"Agent Loop Length: {len(agent.messages)})")
print("\nUser-Assistant Conversation:")
for message in agent.messages:
    print(message)

for message in agent.messages:
    for content in message['content']:
        if 'toolUse' in content and content['toolUse']['input']['code_interpreter_input']['action']['type']=='executeCode':
            code = content['toolUse']['input']['code_interpreter_input']['action']['code']
            print("\nGenerated Code")
            print("--------------")
            print(code)

### 了解沙盒环境限制

默认 Code Interpreter 在具有网络隔离的沙盒环境中运行。这是一个重要的安全功能，可防止未经授权的网络访问，同时确保代码执行安全。

## 创建具有网络访问权限的自定义 Code Interpreter

为了启用基于网络的操作，我们将创建一个具有公共网络访问权限的自定义 Code Interpreter。这展示了 AgentCore 平台在不同用例中的灵活性。

### 步骤 1：初始化 AgentCore 客户端

首先，让我们为控制平面和数据平面操作设置必要的客户端：

In [ ]:
from bedrock_agentcore._utils import endpoints
import boto3
import json

region = boto3.session.Session().region_name

data_plane_endpoint = endpoints.get_data_plane_endpoint(region)
control_plane_endpoint = endpoints.get_control_plane_endpoint(region)

cp_client = boto3.client("bedrock-agentcore-control", 
                        region_name=region,
                        endpoint_url=control_plane_endpoint)

dp_client = boto3.client("bedrock-agentcore", 
                        region_name=region,
                        endpoint_url=data_plane_endpoint)

### 步骤 2：创建自定义 Code Interpreter

创建具有公共网络访问权限的自定义 Code Interpreter：

In [ ]:
import uuid

# Create code interpreter
unique_name = f"sampleCodeInterpreter_{uuid.uuid4().hex[:8]}"
interpreter_response = cp_client.create_code_interpreter(
    name=unique_name,
    description="Environment for Code Interpreter sample test",
    #executionRoleArn=iam_role_arn, #Required only if the code interpreter need to access AWS resources
    networkConfiguration={
        'networkMode': 'PUBLIC'
    }
)
interpreter_id = interpreter_response["codeInterpreterId"]
print(f"Created interpreter: {interpreter_id}")

### 步骤 3：创建 Code Interpreter 会话

在自定义 Code Interpreter 中创建代码解释器会话：

In [ ]:
# Create code interpreter session
session_response = dp_client.start_code_interpreter_session(
    codeInterpreterIdentifier=interpreter_id,
    name="sampleCodeInterpreterSession",
    sessionTimeoutSeconds=900
)
session_id = session_response["sessionId"]
print(f"Created session: {session_id}")

### 步骤 4：测试基本功能

让我们通过执行简单的 "Hello World" 命令来测试代码解释器会话：

In [ ]:
response = dp_client.invoke_code_interpreter(
    codeInterpreterIdentifier=interpreter_id,
    sessionId=session_id,
    name="executeCommand",
    arguments={
        'command': "echo 'Hello World'"
    }
)
for event in response["stream"]:
    print(json.dumps(event["result"], indent=2))

让我们通过使用 pip 安装 DuckDuckGo Python 包来验证代码解释器会话是否可以访问互联网：

In [ ]:
response = dp_client.invoke_code_interpreter(
    codeInterpreterIdentifier=interpreter_id,
    sessionId=session_id,
    name="executeCommand",
    arguments={
        'command': "pip install ddgs"
    }
)
for event in response["stream"]:
    print(json.dumps(event["result"], indent=2))

让我们在代码解释器会话中执行 Python 代码，使用 DuckDuckGo 搜索关键词 "coffee maker amazon.com"：

In [ ]:
response = dp_client.invoke_code_interpreter(
    codeInterpreterIdentifier=interpreter_id,
    sessionId=session_id,
    name="executeCode",
    arguments={"code": """
        from ddgs import DDGS

        keywords = "coffee maker amazon.com"
        results = DDGS().text(keywords, region="us-en", max_results=2)
        print(results)
        """,
        "language": "python",
        "clearContext": False
    }
)
for event in response["stream"]:
    print(json.dumps(event["result"], indent=2))

## 资源清理（可选）

清理我们创建的 AgentCore Runtime 资源以避免不必要的费用：

In [ ]:
import boto3

region = boto3.session.Session().region_name

cp_client = boto3.client("bedrock-agentcore-control", region_name=region, endpoint_url=control_plane_endpoint)
dp_client = boto3.client("bedrock-agentcore", region_name=region, endpoint_url=data_plane_endpoint)

try:
    print("Cleaning up session and interpreter...")
    dp_client.stop_code_interpreter_session(
        codeInterpreterIdentifier=interpreter_id,
        sessionId=session_id
    )
    print("✓ Session stopped successfully")

    cp_client.delete_code_interpreter(codeInterpreterId=interpreter_id)
    print("✓ Interpreter deleted successfully")
except Exception as e:
    print(f"❌ Error during cleanup: {e}")
    print("You may need to manually clean up some resources.")